In [ ]:
import numpy as np
import csv, json
import os, shutil
import cv2

### Extract frame data from videos
Default path: `da_eval/data/frames`

In [ ]:
def csv_label_remapping(in_label:str, label_remapping_dict:dict, id2label:dict):

    """ remap csv label from another dataset to the contact detection purpose set """

    curr_key = label_remapping_dict[in_label]
    label2id = {dct.values[1]: dct.values[0] for dct in id2label}
    return label2id[curr_key]


def vid2imgs(vid_path:str, output_path):

    """ convert video to frame images """

    data_ls = os.listdir(vid_path)
    if not os.path.exists(output_path):
        os.makedirs(output_path)
    vid_ls = [d for d in data_ls if d.endswith('.mp4') ]
    for vid in vid_ls:
        print(f'Start extracting frames from video: {vid} ...')
        vid_name = vid.split('.')[0]
        v_data = cv2.VideoCapture(os.path.join(vid_path, vid))
        count = 0
        while True:
            success, frame = v_data.read()
            if not success:
                print('Done.')
                break
            save_path_f = os.path.join(output_path, f'{vid_name}_{count}.png')
            cv2.imwrite(save_path_f, frame)
            count += 1
                

vid2imgs('da_eval/data/mtm_augmented_data',
            'da_eval/data/frames')

### Convert keypoint annotations from `csv` to bounding box in `json` 

In [ ]:
def csvAnn2JsonConverterSingle(csv_path:str, criterion:str, categories:dict): 
    
    """ Convert a single csv annotation file to json format """
    
    csvfile = open(csv_path, 'r')
    a_table = csv.reader(csvfile)
    csvfile.close
    feas_crit = [None, 'handstate']
    if criterion not in feas_crit:
        raise KeyError('Given criterion is not a feasible one!')

    imgInfos = {
        "id": [],
        "width": [],
        "height": [],
        "file_name": []
    }
    label_remapping_dict_hand = {
        'Negative': 'background',
        'Move': 'hand',
        'Release': 'hand',
        'Position': 'hand',
        'Grasp': 'hand',
        'Reach': 'hand',
    }
    label_remapping_dict_handstate = {
        'Negative': 'None',
        'Move': 'hand contacting with a portable object',
        'Release': 'hand contacting with a portable object',
        'Position': 'hand contacting with a portable object',
        'Grasp': 'hand contacting with a portable object',
        'Reach': 'hand with no contact',
    }
    ann = {
        "id": [],
        "category_id": [],
        "iscrowd": [],
        "image_id": [],
        "area": [],
        "bbox": []
    },

    anns, im_infos = [], []
    width, height = [], []

    count = 0
    for a in a_table:
        idx = int(a[0])
        file_name = f'{idx}.jpg'
        if count == 0:
            img = cv2.imread(file_name)
            width, height = img.shape[:-2]
        
        label = a[-1]
        keypts = [[float(pts[1]) * width, float(pts[3]) * height] for pts in a[-1:1]]    # pts[1]: x, pts[3]: y
        keypts = np.asanyarray(keypts)
        x_min, y_min = np.min(keypts, axis=-1).astype(np.int)
        x_max, y_max = np.max(keypts, axis=-1).astype(np.int)
        box = [x_min, y_min, x_max, y_max]
        
        if criterion is None:
            label_remapping_dict = label_remapping_dict_hand
            category_id = csv_label_remapping(label, label_remapping_dict, categories)
        elif criterion == 'handstate':
            label_remapping_dict = label_remapping_dict_handstate
            category_id = csv_label_remapping(label, label_remapping_dict, categories)

        imgInfos["file_name"] = file_name
        imgInfos["height"] = height
        imgInfos['width'] = width
        imgInfos["id"] = idx

        ann['id'] = idx
        ann['category_id'] = category_id
        ann['is_crowd'] = 0
        ann['image_id'] = idx
        ann['area'] = int((x_max - x_min) * (y_max - y_min))
        ann['bbox'] = box

        im_infos.append(imgInfos)
        anns.append(ann)

    return im_infos, anns

def csv_json_convertor(csv_path, output_path, criterion):

    """ 
    CSV format: mediapipe keypoints, class -> [x, y, z]:float * 18, 1:str
    json format: {
        info: [],
        license: [],
        categories: [], # id to label
        images: [], image infos
        annotations: [], annotations
    }
    """

    # initialize dict with key and empty value pair
    annDict = {'info':{}, # null
               'licenses': [], # null
               'categories': [], # containing categories and class numbers 
               'images':[], # containing file path, id and size
               'annotations': [] # containing annotations
               }

    # set map between ids and categories 
    if criterion is not None:
        if criterion == 'handside':
            categories = [
                {"id":0,"name":"targetobject"},
                {"id":1,"name":"lefthand"},
                {"id":2,"name":"righthand"},
            ]
        elif criterion == 'handstate':
            categories = [
                {"id":0,"name":"targetobject"},
                {"id":1,"name":"hand with no contact"},
                {"id":2,"name":"hand with self contact"},
                {"id":3,"name":"hand contacting with other person"},
                {"id":4,"name":"hand contacting with a portable object"},
                {"id":5,"name":"hand contacting with a static object"},
            ]
        else:
            raise ValueError("The given criterion not supported!")
    else:
        categories = [
                {"id":0,"name":"hand"},
                {"id":1,"name":"targetobject"},
                {"id":2,"name":"background"},
            ]
    
    annDict["categories"] = categories

    csv_ann_ls = [f for f in os.listdir(csv_path) if f.endswith('original.csv')]
    for f in csv_ann_ls:
        vid_id = int(f[0])
        full_path_ann = os.path.join(csv_path, f)
        im_infos, ann = csvAnn2JsonConverterSingle(full_path_ann, criterion, categories)
        
        annDict['images'] = im_infos
        annDict['annotations'] = ann
        save_path = os.path.join(output_path, vid_id)
        if not os.path.exists(save_path):
            raise NotImplementedError("Bug here!!!")
        save_path = os.path.join(save_path, f'{criterion}.json' if criterion is not None else 'hand.json')
        with open(save_path, 'w') as f:
            json.dump(annDict, f, indent=4)

In [ ]:
criterion = None

csv_json_convertor(csv_path="da_eval/data/mtm_augmented_data",
                   output_path="da_eval/data/frames",
                   criterion=criterion)